In [1]:
import plotly.express as px
import json
import pandas as pd
from glob import glob

In [4]:
jsons = glob("./stats/*/*.json")

training_steps = []
datasets = []
initial_r2 = []
final_r2 = []
test_acc = []
datasets = []
l = []
l2 = []
l3 = []

for file in jsons:
    # print(file)
    try:
        j = json.load(open(file, "r"))
        l.append(j["benford_list"])
        l2.append(j["val_acc_step"])
        l3.append(j["train_acc_step"])
        initial_r2.append(j['initial_r2'])
        final_r2.append(j['final_r2'])
        test_acc.append(j['test_acc'])
        training_steps.append(j['training_steps'])
        datasets.append(j['dataset'])
    except:
        # print(f"skipping {file}")
        pass
print("Total JSONS: ", len(jsons))
df = pd.DataFrame(zip(datasets, initial_r2, final_r2, test_acc, training_steps, l, l2, l3), 
                       columns=['dataset', 'initial_r2', 'final_r2', 'test_acc', 'training_steps', 'benford_list', 'val_acc_step', 'train_acc_step'])

print(df['dataset'].value_counts())
df

Total JSONS:  1873
fashion     200
aircraft     15
flowers      14
dogs         12
cifar10       1
mnist         1
Name: dataset, dtype: int64


,dataset,initial_r2,final_r2,test_acc,training_steps,benford_list,val_acc_step,train_acc_step
0,aircraft,0.969462,0.995249,0.427024,4316,"[0.9694620895606703, 0.9694202248359498, 0.969...","[{'validation_accuracy': 0.0, 'step': 0}, {'va...","[{'training_acc': 0.0, 'step': 1}, {'training_..."
1,aircraft,0.969177,0.995721,0.458869,4384,"[0.9691769380943837, 0.9691862559382092, 0.969...","[{'validation_accuracy': 0.0, 'step': 0}, {'va...","[{'training_acc': 0.0625, 'step': 1}, {'traini..."
2,aircraft,0.969469,0.994496,0.385774,4175,"[0.9694691732140002, 0.9694029998922824, 0.969...","[{'validation_accuracy': 0.0, 'step': 0}, {'va...","[{'training_acc': 0.0, 'step': 1}, {'training_..."
3,aircraft,0.969045,0.994937,0.523571,4734,"[0.9690449534663836, 0.9691193471001417, 0.969...","[{'validation_accuracy': 0.0, 'step': 0}, {'va...","[{'training_acc': 0.0, 'step': 1}, {'training_..."
4,aircraft,0.969092,0.994526,0.531190,4525,"[0.9690915167561198, 0.9691581517774036, 0.969...","[{'validation_accuracy': 0.0, 'step': 0}, {'va...","[{'training_acc': 0.0, 'step': 1}, {'training_..."
...,...,...,...,...,...,...,...,...
238,fashion,0.994486,0.972127,0.897035,14027,"[0.9946973023027225, 0.7799497384564715, 0.738...","[{'validation_accuracy': 0.140625, 'step': 0},...","[{'training_acc': 0.0625, 'step': 1}, {'traini..."
239,fashion,0.994079,0.966090,0.885216,14027,"[0.9941778777533548, 0.778603298559786, 0.8602...","[{'validation_accuracy': 0.078125, 'step': 0},...","[{'training_acc': 0.078125, 'step': 1}, {'trai..."
240,fashion,0.994414,0.971908,0.894832,14027,"[0.9944122466969689, 0.7815955831202291, 0.727...","[{'validation_accuracy': 0.0390625, 'step': 0}...","[{'training_acc': 0.09375, 'step': 1}, {'train..."
241,mnist,0.953095,0.952326,0.987580,29772,"[0.9939751479765937, 0.7825811727770666, 0.824...","[{'validation_accuracy': 0.1328125, 'step': 0}...","[{'training_acc': 0.15625, 'step': 1}, {'train..."


In [111]:
import random
import plotly.graph_objects as go
import numpy as np

def smooth(y, box_pts):
    box = np.ones(box_pts)/box_pts
    y_smooth = np.convolve(y, box, mode='same')
    return y_smooth
    
unique = df.dataset.unique()
fig = go.Figure()
color = dict(
    mnist="#f9de59",
    dogs="#e8a628",
    aircraft="#dcec94",
    cifar10="#a1dffb",
    fashion="#c33124",
    flowers="#FF69B4",
)
names = dict(
    mnist="MNIST",
    dogs="Dogs",
    aircraft="Aircrafts",
    cifar10="CIFAR10",
    fashion="FMNIST",
    flowers="Flowers",
)
for dataset in unique:
    # idx = df.loc[df.dataset == dataset].index
    # sampled = random.choice(idx)
    # print(dataset, sampled)
    mlh = np.array(df['benford_list'][indices[dataset]])
    if dataset in ["mnist", "cifar", "fashion"]:
        mlh = smooth(mlh, 50)
    # assert ~np.allclose(mlh, mlh_)
    fig.add_trace(
            go.Scatter(
                y=mlh,
                name=names[dataset],
                marker_color=color[dataset]
        )
    )

fig.update_layout(
    yaxis={"range":[0.97, 1.0]},
    xaxis={"range":[0, 10000]},
)
fig.update_layout(font=dict(
    family="Times New Roman",
    size=20,
    color="Black",), autosize=True
)

fig.update_layout(template='none')
fig.update_xaxes(title_text="Training Steps", showline=True, mirror=True, showgrid=True)
fig.update_yaxes(title_text="MLH", automargin=True, dtick=0.01, showline=True, mirror=True, showgrid=True)
fig.update_layout(
    autosize=False,
    # width=1500,
    height=370,)
fig.update_traces(marker=dict(size=2))
fig.show()

In [1]:
import numpy as np
import plotly.express as px
from functools import partial
from scipy import stats
from joblib import Parallel, delayed
from numba import njit
import plotly.graph_objects as go
import pandas as pd


benford = np.array([30.1,
                    17.6,
                    12.5,
                    9.7,
                    7.9,
                    6.7,
                    5.8,
                    5.1,
                    4.6]) / 100

# @njit
def benford_r2(bin_percent):
    return stats.pearsonr(benford, bin_percent[1:])[0]

import torch
# @njit
def bincount(tensor):
    counts = torch.zeros(10)
    for i in range(10):
        counts[i] = torch.count_nonzero(tensor == i)
    return counts


@torch.no_grad()
def bin_percent(tensor):
    tensor = torch.tensor(tensor)
    tensor = tensor.abs() * 1e10
    tensor = tensor // 10 ** torch.log10(tensor).long()
    tensor = bincount(tensor.long())
    return (tensor / tensor.sum()).numpy()
from numba import njit

@njit
def boltzmann_gibbs(beta, E):
    """Returns the probability of an Ideal Gas in a seal container system at temperature T will have energy E

    Args:
        T (float): Temperature of the Ideal Gas
        E (float): Energy of the Ideal Gas

    Returns:
        float: Probability
    """
    return beta * np.exp(-beta * E)

@njit
def fermi_dirac(E, beta):
    """Returns the probability of an Ideal Gas in a seal container system at temperature T will have energy E

    Args:
        T (float): Temperature of the Ideal Gas
        E (float): Energy of the Ideal Gas

    Returns:
        float: Probability
    """
    # print(beta)
    return (beta/np.log(2)) / (1 + np.exp(beta * E))

class FermiDirac(stats.rv_continuous):
    def _pdf(self, E, beta):
        return (beta/np.log(2)) / (1 + np.exp(beta * E))


@njit
def icdf_bg(x, beta):
    '''Inverse Cumulative Distribution Function for Boltzmann-Gibbs'''
    return -np.log(x)/beta

@njit
def icdf_fd(x, beta):
    '''This doesn't work, probably a mistake in integrating and taking inverse'''
    #print(x, 2 ** -x)
    return (-1/beta) * np.log(2. ** (-x) - 1)

def get_beta_i(beta_i, num_vals):
    samples = icdf_bg(
        np.random.uniform(size=num_vals),
        beta_i
    )
    mlh = benford_r2(bin_percent(samples))
    # print(mlh)
    return mlh

from tqdm.notebook import tqdm
def generate_samples():
    # beta = np.logspace(-0.1, -1, base=0.1, num=10000, endpoint=True)
    beta = np.linspace(0.1, 10.0, num=10000)
    num_vals = int(2**15)
    # fig = go.Figure()

    mlh = Parallel(n_jobs=-1)(delayed(get_beta_i)(b, num_vals) for b in tqdm(beta))
    # fig.add_trace(
    #         go.Scatter(
    #             y=mlh,
    #             # name=names[dataset],
    #             marker_color="#CE9DD9",
    #             mode="markers"
    #     )
    # )
    # color=["#CE9DD9"]*len(beta)
    # mlh = np.log(mlh)
    df=pd.DataFrame({"Beta":beta, "MLH":mlh})
    fig=px.scatter(df, x="Beta", y="MLH")
    fig.update_layout(font=dict(
    family="Times New Roman",
    size=20,
    color="Black",), autosize=True)
    fig.update_layout(template='none',)
    fig.update_xaxes(title_text="1/kT", automargin=True,)
    fig.update_yaxes(title_text="MLH", automargin=True)
    fig.update_xaxes(mirror=True, ticks="outside", showline=True)
    fig.update_yaxes(mirror=True, ticks="outside", showline=True, tick0=0.98, dtick=0.004)
    fig.update_layout(
    # yaxis={"range":[0.97, 1.0]},
    xaxis={"range":[0.1, 10]},
)
    fig.update_layout(
    autosize=False,
    # width=1500,
    height=300,)
    fig.update_traces(marker=dict(size=2))
    fig.show()

def plot_pdf():
    space = np.linspace(0.0, 1000000, num=10000*10)
    probs = [boltzmann_gibbs(1, e) for e in space]
    fig = px.line(x=space, y=probs)

    fig.show()

if __name__ == "__main__":
    generate_samples()
    

  0%|          | 0/10000 [00:00<?, ?it/s]